# NODE

In [ ]:
pip install pandas scikit-learn openml torch numpy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.5 MB/s eta 0:00:00
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11717 sha256=2874b705016a9d4192b8b7c53b93995abf0362fcafef38f2a627560255a23b09
  Stored in directory: /root/.cache/pip/wheels/5d/2a/9c/3895d9617f8f49a0883ba686326d598e78a1c2f54fe3cae86d
Successfully built liac-arff


In [ ]:
pip install torch_optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.7 MB/s eta 0:00:00


In [ ]:
import time
import shutil
import numpy as np
import openml
import pandas as pd  # Make sure to import pandas
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim  # Importing standard PyTorch optimizers
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import os

# Set a custom OpenML cache directory
openml.config.cache_directory = os.path.expanduser("~/.openml_cache")

class NODE(nn.Module):
    def __init__(self, params, num_features, num_classes):
        super(NODE, self).__init__()

        layer_dim = int(params["total_tree_count"] / params["num_layers"])

        self.model = nn.Sequential(
            nn.Linear(num_features, layer_dim),  # Example layer, modify as needed
            nn.ReLU(),
            nn.Linear(layer_dim, num_classes)
        )

    def forward(self, x):
        return self.model(x)

    def predict(self, X):
        self.eval()
        with torch.no_grad():
            return self.forward(X)

def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

def get_random_parameters(seed):
    rs = np.random.RandomState(seed)
    params = {
        "num_layers": rs.choice([2, 4, 8]),
        "total_tree_count": rs.choice([1024, 2048]),
        "tree_depth": rs.choice([6, 8]),
        "tree_output_dim": rs.choice([2, 3]),
        "learning_rate": 1e-4,  # Smaller learning rate
        "epochs": 100,  # Example value, set as needed
        "objective": 'classification',  # Assuming classification
    }
    return params

def load_data(task_id):
    task = openml.tasks.get_task(task_id)
    dataset = openml.datasets.get_dataset(task.dataset_id)

    # Retrieve the target feature directly from the task
    target_feature = task.target_name  # Changed from task.get_target() to task.target_name

    # Retrieve the dataset's data
    X, y, _, _ = dataset.get_data(target=target_feature)
    return X, y

def preprocess_data(X):
    # Convert categorical columns to numeric using one-hot encoding
    if isinstance(X, pd.DataFrame):
        X = pd.get_dummies(X, drop_first=True)  # One-hot encode categorical variables

    # Handle NaN values by filling them with the column mean or dropping rows
    X = X.fillna(X.mean())  # Fill NaNs with column mean for numeric stability
    return X

def encode_target(y):
    # Use LabelEncoder to convert categorical target variable to numeric
    le = LabelEncoder()
    return le.fit_transform(y)

def train_model(X_train, y_train, X_val, y_val, params):
    num_features = X_train.shape[1]
    num_classes = len(np.unique(y_train))

    model = NODE(params, num_features, num_classes).to(device)

    if params['objective'] == 'classification':
        loss_function = nn.CrossEntropyLoss()
    elif params['objective'] == 'regression':
        loss_function = nn.MSELoss()

    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])  # Use standard PyTorch Adam

    # Check for NaN values in input data using Pandas method
    if X_train.isna().any().any() or X_val.isna().any().any():
        raise ValueError("Input data contains NaN values.")

    # Normalize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)

    # Initialize model weights
    model.apply(initialize_weights)

    # Training loop
    for epoch in range(params['epochs']):
        model.train()
        optimizer.zero_grad()

        # Convert DataFrame to FloatTensor
        outputs = model(torch.FloatTensor(X_train).to(device))  # Convert to NumPy array
        loss = loss_function(outputs, torch.LongTensor(y_train).to(device))  # Ensure y_train is LongTensor for classification
        loss.backward()
        optimizer.step()

        if np.isnan(loss.item()):  # Check for NaN loss
            print("NaN loss encountered, stopping training.")
            break

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(torch.FloatTensor(X_val).to(device))  # Convert to NumPy array
        val_predictions = torch.argmax(val_outputs, dim=1)
        accuracy = accuracy_score(y_val, val_predictions.cpu())

    return model, accuracy

# Define your parameters
params = get_random_parameters(seed=42)



# Task ID1: 14965

In [ ]:
# Load the data from OpenML
task_id = 14965  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8795


# Task ID2: 9977

In [ ]:
# Load the data from OpenML
task_id = 9977  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9179


# Task ID3: 34539

In [ ]:
# Load the data from OpenML
task_id = 34539  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9179


# Task ID4: 146606

In [ ]:
# Load the data from OpenML
task_id = 146606  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5961


# Task ID5: 7592



In [ ]:
# Load the data from OpenML
task_id = 7592  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8271


# Task ID6: 146195



In [ ]:
# Load the data from OpenML
task_id = 146195  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7077


# Task ID7: 167119



In [ ]:
# Load the data from OpenML
task_id = 167119  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6431


# Task ID8: 167120



In [ ]:
# Load the data from OpenML
task_id = 167120  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5103


# Task ID11: 	168331



In [ ]:
# Load the data from OpenML
task_id = 168331  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5065


# Task ID12: 	168330



In [ ]:
# Load the data from OpenML
task_id = 168330  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5555


# Task ID13: 	168335



In [ ]:
# Load the data from OpenML
task_id = 168335  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8076



# Task ID16: 	146212



In [ ]:
# Load the data from OpenML
task_id = 146212  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8828


# Task ID19: 	168868



In [ ]:
# Load the data from OpenML
task_id = 168868  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9825


# Task ID20: 31

In [ ]:
# Load the data from OpenML
task_id = 31  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7300


# Task ID21: 10101

In [ ]:
# Load the data from OpenML
task_id = 10101  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7533


# Task ID22: 	3913

In [ ]:
# Load the data from OpenML
task_id = 3913  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8667


# Task ID23: 3

In [ ]:
# Load the data from OpenML
task_id = 3  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7391


# Task ID24: 3917

In [ ]:
# Load the data from OpenML
task_id = 3917  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8507


# Task ID25: 9957

In [ ]:
# Load the data from OpenML
task_id = 9957 # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8578


# Task ID26: 9946

In [ ]:
# Load the data from OpenML
task_id = 9946  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9561


# Task ID27: 3918

In [ ]:
# Load the data from OpenML
task_id = 3918  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9144


# Task ID28: 3903

In [ ]:
# Load the data from OpenML
task_id = 3903  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8946


# Task ID29: 37

In [ ]:
# Load the data from OpenML
task_id = 37  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7532


# Task ID30: 9971

In [ ]:
# Load the data from OpenML
task_id = 9971  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7521


# Task ID31: 9952

In [ ]:
# Load the data from OpenML
task_id = 9952  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7771


# Task ID32: 3902

In [ ]:
# Load the data from OpenML
task_id = 3902  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8493


# Task ID33: 49

In [ ]:
# Load the data from OpenML
task_id = 49  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7083


# Task ID34: 43

In [ ]:
# Load the data from OpenML
task_id = 43  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8697


# Task ID35: 9978

In [ ]:
# Load the data from OpenML
task_id = 9978  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9389


# Task ID36: 10093

In [ ]:
# Load the data from OpenML
task_id = 10093  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6509


# Task ID37: 219

In [ ]:
# Load the data from OpenML
task_id = 219  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6895


# Task ID38: 9976

In [ ]:
# Load the data from OpenML
task_id = 9976  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5404


# Task ID39: 6

In [ ]:
# Load the data from OpenML
task_id = 6  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.2350


# Task ID40: 53

In [ ]:
# Load the data from OpenML
task_id = 53  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5353


# Task ID41: 11

In [ ]:
# Load the data from OpenML
task_id = 11  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8080


# Task ID42: 15

In [ ]:
# Load the data from OpenML
task_id = 15  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9714


# Task ID43: 16

In [ ]:
# Load the data from OpenML
task_id = 16  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6950


# Task ID44: 14

In [ ]:
# Load the data from OpenML
task_id = 14  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5350


# Task ID45: 32

In [ ]:
# Load the data from OpenML
task_id = 32  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6221


# Task ID46: 3549

In [ ]:
# Load the data from OpenML
task_id = 3549  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9527


# Task ID47: 12

In [ ]:
# Load the data from OpenML
task_id = 12  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9175


# Task ID48: 9981

In [ ]:
# Load the data from OpenML
task_id = 9981  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8102


# Task ID49: 18

In [ ]:
# Load the data from OpenML
task_id = 18  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.3325


# Task ID50: 28

In [ ]:
# Load the data from OpenML
task_id = 28  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7402


# Task ID51: 2074

In [ ]:
# Load the data from OpenML
task_id = 2074  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7729


# Task ID52: 29

In [ ]:
# Load the data from OpenML
task_id = 29  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7971


# Task ID53: 45

In [ ]:
# Load the data from OpenML
task_id = 45  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7461


# Task ID54: 125922

In [ ]:
# Load the data from OpenML
task_id = 125922  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6091


# Task ID55: 9960

In [ ]:
# Load the data from OpenML
task_id = 9960  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5879


# Task ID56: 9964

In [ ]:
# Load the data from OpenML
task_id = 9964  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7962


# Task ID57: 22

In [ ]:
# Load the data from OpenML
task_id = 22  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6225


# Task ID58: 2079

In [ ]:
# Load the data from OpenML
task_id = 2079  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5270


# Task ID59: 14969

In [ ]:
# Load the data from OpenML
task_id = 14969  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.4608


# Task ID60: 3560

In [ ]:
# Load the data from OpenML
task_id = 3560  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.2125


# Task ID61: 14952

In [ ]:
# Load the data from OpenML
task_id = 14952  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8123


# Task ID62: 125920

In [ ]:
# Load the data from OpenML
task_id = 125920  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5400


# Task ID63: 23

In [ ]:
# Load the data from OpenML
task_id = 23  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.4576


# Task ID64: 3904

In [ ]:
# Load the data from OpenML
task_id = 3904  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8130


# Task ID65: 3022

In [ ]:
# Load the data from OpenML
task_id = 3022  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.2475


# Task ID66: 9985

In [ ]:
# Load the data from OpenML
task_id = 9985  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.4044


# Task ID67: 9910

In [ ]:
# Load the data from OpenML
task_id = 9910  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7790


# Task ID68: 14970

In [ ]:
# Load the data from OpenML
task_id = 14970  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9170


# Task ID69: 3021

In [ ]:
# Load the data from OpenML
task_id = 3021  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9497


# Task ID70: 3481

In [ ]:
# Load the data from OpenML
task_id = 3481  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8154


# Task ID71: 3573

In [ ]:
# Load the data from OpenML
task_id = 3573  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


# Task ID72: 146824

In [ ]:
# Load the data from OpenML
task_id = 146824  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9300


# Task ID73: 146820

In [ ]:
# Load the data from OpenML
task_id = 146820 # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9473


# Task ID74: 146822

In [ ]:
# Load the data from OpenML
task_id = 146822  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5455


# Task ID75: 146195

In [ ]:
# Load the data from OpenML
task_id = 146195  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6934


# Task ID76: 146800

In [ ]:
# Load the data from OpenML
task_id = 146800  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6481


# Task ID77: 146817

In [ ]:
# Load the data from OpenML
task_id = 146817  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.5784


# Task ID78: 146819

In [ ]:
# Load the data from OpenML
task_id = 146819  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9074


# Task ID79: 146821

In [ ]:
# Load the data from OpenML
task_id = 146821  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.6821


# Task ID80: 14954

In [ ]:
# Load the data from OpenML
task_id = 14954  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.7963


# Task ID81: 167141

In [ ]:
# Load the data from OpenML
task_id = 167141  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8510


# Task ID82: 167140

In [ ]:
# Load the data from OpenML
task_id = 167140  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.8009


# Task ID83: 167125

In [ ]:
# Load the data from OpenML
task_id = 167125  # Replace with your OpenML task ID
X, y = load_data(task_id)

# Preprocess the data
X = preprocess_data(X)

# Encode the target variable
y = encode_target(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Train the model
model, accuracy = train_model(X_train, y_train, X_val, y_val, params)
print(f'Validation Accuracy: {accuracy:.4f}')


Validation Accuracy: 0.9771


# Task ID84: 167124

# Task ID85: 167121

In [ ]:
pip freeze > requirements.txt